## Select 심화

오늘은 기본적으로 `select` 를 이용하는 내용이 아닌 조금 더 딥하게 다뤄보겠습니다. 

In [2]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session

class Base(DeclarativeBase):
    """
    DeclarativeBase 를 상속 받은 Base 라는 하위 클래스를 만들고 시작.
    이 Base 에 Mapping 된 클래스들은 database 에서 단일 테이블임.

    `__tablename__` 을 클래스 레벨의 속성으로 지녀야 함.
    """
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

Base.metadata.create_all(engine)  
    

2025-04-30 21:51:47,134 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-30 21:51:47,135 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-04-30 21:51:47,135 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-30 21:51:47,136 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-04-30 21:51:47,137 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-30 21:51:47,138 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-04-30 21:51:47,138 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-30 21:51:47,139 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2025-04-30 21:51:47,139 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-30 21:51:47,140 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2025-04-30 21:51:47,141 INFO sqlalchemy.engine.Engine [no key 0.00035s] ()
2025-04-30 21:51:47,142 INFO sqlalchemy.engine.

In [6]:
from sqlalchemy.orm import Session

def generate_100_users():
    with Session(engine) as session:
        for i in range(100):
            session.add(User(name=f"user{i}", fullname=f"User {i}", addresses=[Address(email_address=f"user{i}@example.com")]))
        session.flush()
        session.commit()

generate_100_users()  

2025-04-30 21:53:35,576 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-30 21:53:35,578 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-04-30 21:53:35,579 INFO sqlalchemy.engine.Engine [cached since 9.673s ago (insertmanyvalues) 1/100 (ordered; batch not supported)] ('user0', 'User 0')
2025-04-30 21:53:35,579 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-04-30 21:53:35,580 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/100 (ordered; batch not supported)] ('user1', 'User 1')
2025-04-30 21:53:35,581 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-04-30 21:53:35,581 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/100 (ordered; batch not supported)] ('user2', 'User 2')
2025-04-30 21:53:35,582 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-04-30 21:53:35,5

## Result 와 Row

SQLAlchemy 의 공식문서를 읽어보면 Session.execute() 는 2.0 버전에서 ORM 에서 statement 를 실행시키기 위해 알아두어야 할 중요할점이라고 적어두었는데요. 문서를 읽어보면 `Session.execute()` 에서는 `Result` 오브젝트를 반환한다고 적혀있습니다. 중요한 만큼 `Result` 에 대해서도 알아봐야 할거 같은데요. 과연 `Result` 객체는 어떤 역할을 할까요?

In [27]:
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User)
    users = session.execute(stmt)
    print(users) # sqlalchemy.engine.result.ChunkedIteratorResult
    print(type(users)) # class 'sqlalchemy.engine.result.ChunkedIteratorResult'

2025-04-30 22:29:18,758 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-30 22:29:18,760 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-04-30 22:29:18,761 INFO sqlalchemy.engine.Engine [cached since 1981s ago] ()
<class 'sqlalchemy.engine.result.ChunkedIteratorResult'>
2025-04-30 22:29:18,762 INFO sqlalchemy.engine.Engine ROLLBACK


실제로 위 코드를 실행해보니 `ChunkedIteratorResult` 타입의 객체를 반환합니다. 이 객체는 무엇일까요? 공식문서에서는 아래와 같이 설명하고 있습니다.
사실 `Result` 자체는 쿼리 수행으로 얻어지는 추상적인 표현입니다. 정확히 보기 위해 실제 코드를 잠시 가져와 보겠습니다.

```python
class Result(_WithKeys, ResultInternal[Row[Unpack[_Ts]]]):
    """Represent a set of database results.

    .. versionadded:: 1.4  The :class:`_engine.Result` object provides a
       completely updated usage model and calling facade for SQLAlchemy
       Core and SQLAlchemy ORM.   In Core, it forms the basis of the
       :class:`_engine.CursorResult` object which replaces the previous
       :class:`_engine.ResultProxy` interface.   When using the ORM, a
       higher level object called :class:`_engine.ChunkedIteratorResult`
       is normally used.

    .. note:: In SQLAlchemy 1.4 and above, this object is
       used for ORM results returned by :meth:`_orm.Session.execute`, which can
       yield instances of ORM mapped objects either individually or within
       tuple-like rows. Note that the :class:`_engine.Result` object does not
       deduplicate instances or rows automatically as is the case with the
       legacy :class:`_orm.Query` object. For in-Python de-duplication of
       instances or rows, use the :meth:`_engine.Result.unique` modifier
       method.

    .. seealso::

        :ref:`tutorial_fetching_rows` - in the :doc:`/tutorial/index`

    """

    __slots__ = ("_metadata", "__dict__")

    _row_logging_fn: Optional[
        Callable[[Row[Unpack[TupleAny]]], Row[Unpack[TupleAny]]]
    ] = None

    _source_supports_scalars: bool = False

    _yield_per: Optional[int] = None

    _attributes: util.immutabledict[Any, Any] = util.immutabledict()

    def __init__(self, cursor_metadata: ResultMetaData):
        self._metadata = cursor_metadata

    def __enter__(self) -> Self:
        return self

    def __exit__(self, type_: Any, value: Any, traceback: Any) -> None:
        self.close()

    def close(self) -> None:
        """close this :class:`_engine.Result`.

        The behavior of this method is implementation specific, and is
        not implemented by default.    The method should generally end
        the resources in use by the result object and also cause any
        subsequent iteration or row fetching to raise
        :class:`.ResourceClosedError`.

        .. versionadded:: 1.4.27 - ``.close()`` was previously not generally
           available for all :class:`_engine.Result` classes, instead only
           being available on the :class:`_engine.CursorResult` returned for
           Core statement executions. As most other result objects, namely the
           ones used by the ORM, are proxying a :class:`_engine.CursorResult`
           in any case, this allows the underlying cursor result to be closed
           from the outside facade for the case when the ORM query is using
           the ``yield_per`` execution option where it does not immediately
           exhaust and autoclose the database cursor.

        """
        self._soft_close(hard=True)

    @property
    def _soft_closed(self) -> bool:
        raise NotImplementedError()

    @property
    def closed(self) -> bool:
        """return ``True`` if this :class:`_engine.Result` reports .closed

        .. versionadded:: 1.4.43

        """
        raise NotImplementedError()
```

In [17]:
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User.name, User.fullname)
    users = session.execute(stmt).all()
    print(type(users)) # list
    print(type(users[0])) # 공식문서에서는 named_tuple 이라고 표현하지만, functional 하게만 tuple 처럼 런타임에 동작하고 실상 타입은 Row 임

2025-04-30 22:07:32,147 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-30 22:07:32,149 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2025-04-30 22:07:32,149 INFO sqlalchemy.engine.Engine [cached since 112.2s ago] ()
<class 'list'>
<class 'sqlalchemy.engine.row.Row'>
2025-04-30 22:07:32,152 INFO sqlalchemy.engine.Engine ROLLBACK


In [24]:
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User.name, User.fullname).where(User.name == "user1")
    users = session.execute(stmt).tuples().all()
    print(type(users)) # list
    print(type(users[0])) 

2025-04-30 22:09:40,325 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-30 22:09:40,327 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-04-30 22:09:40,327 INFO sqlalchemy.engine.Engine [cached since 74.31s ago] ('user1',)
<class 'list'>
<class 'sqlalchemy.engine.row.Row'>
2025-04-30 22:09:40,330 INFO sqlalchemy.engine.Engine ROLLBACK
